In [ ]:
import torch
import torchlensmaker as tlm
from torch.func import grad

def sample_cylinder(
    N: int, M: int,
    xmin: torch.Tensor,
    xmax: torch.Tensor,
    rmin: torch.Tensor,
    rmax: torch.Tensor,
    dtype: torch.dtype,
) -> torch.Tensor:
    """
    Generate a grid of points within a cylinder defined by xmin, xmax, and tau.
    """
    theta = torch.linspace(0, 2 * torch.pi, N, dtype=dtype)
    r = torch.linspace(rmin, rmax, N, dtype=dtype)
    x = torch.linspace(xmin, xmax, M, dtype=dtype)

    X, R, Theta = torch.meshgrid(x, r, theta, indexing="xy")

    Y = R * torch.cos(Theta)
    Z = R * torch.sin(Theta)
    
    grid = torch.stack((X, Y, Z), dim=-1).reshape(-1, 3)
    return grid


xmin, xmax, tau = torch.tensor([-50., 50., 30.]).unbind(-1)

points = sample_cylinder(12, 6, xmin-1.0, xmax+1.0, 0.9*tau, 1.1*tau, dtype=torch.float64)

rod = tlm.ImplicitCylinder(xmin, xmax, tau)
sdf = rod.F(points)

fpoints = points[sdf >=0]

# Setup scene for rendering
scene = tlm.new_scene("3D")

# add a dummy surface with a hard codede cylinder for size reference
bcyl = [xmin.item(), xmax.item(), tau.item()]
scene["data"].append({"type": "surface-sag", "diameter": 10, "sag-function": {"sag-type": "spherical", "C": 0.15384615384615385}, "bcyl": bcyl, "matrix": [[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]})

# Render points
scene["data"].append(tlm.render_points(fpoints))

# Display
scene["controls"] = {"show_bounding_cylinders": True, "show_optical_axis": True, "show_other_axes": True}
tlm.display_scene(scene)

In [ ]:
import torch
from torchlensmaker.surfaces.implicit_surface import ImplicitSurface
from torchlensmaker.core.collision_detection import LM

def find_closest_points(surface, P, V):
    N = P.shape[0]
    
    t = torch.zeros(N).unsqueeze(0)
    algo = LM(0.5)

    for i in range(30):
        t = t - algo.delta(surface, P, V, t, max_delta=100.0)

    return t.squeeze(0)


points = torch.rand((10, 3)).unsqueeze(0)
rod = tlm.ImplicitCylinder(torch.tensor(-50.0 / 2.0, dtype=torch.float64),
                           torch.tensor(50.0 / 2.0, dtype=torch.float64),
                           torch.tensor(37.02 / 2.0, dtype=torch.float64),
)

print(rod.F_grad(points))

In [ ]:
import torchlensmaker as tlm
import json

def make_random_rays(num_rays, start_x, end_x, max_y, dtype):
    rays_start = (torch.rand((num_rays, 3), dtype=dtype) * 2 - 1) * max_y
    rays_start[:, 0] = start_x

    rays_end = (torch.rand((num_rays, 3)) * 2 - 1) * max_y
    rays_end[:, 0] = end_x

    rays_vectors = torch.nn.functional.normalize(rays_end - rays_start, dim=1)

    return torch.hstack((rays_start, rays_vectors))

def demo():
    tau = torch.tensor(37.02 / 2.0, dtype=torch.float64)
    xmin = torch.tensor(-50.0 / 2.0, dtype=torch.float64)
    xmax = -xmin

    test_rays = make_random_rays(
        num_rays=30,
        start_x=xmin,
        end_x=xmax,
        max_y=2*tau,
        dtype=torch.float64
    )
    ray_render_start = xmin - 30
    ray_render_end = xmax + 40

    P, V = test_rays[:, :3], test_rays[:, 3:6]

    surface = tlm.ImplicitCylinder(xmin, xmax, tau)
    
    t = find_closest_points(surface, P, V)
    points = P + t.unsqueeze(-1).expand_as(V) * V
    F = surface.F(points)
    print("loss:", F.sum().item())

    # Setup scene for rendering
    scene = tlm.new_scene("3D")
    
    # add a dummy surface with a hard coded cylinder for size reference
    bcyl = [xmin.item(), xmax.item(), tau.item()]
    scene["data"].append({"type": "surface-bcyl", "bcyl": bcyl, "matrix": [[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]})
    
    # Render points
    p = tlm.render_points(points, radius=0.5)
    scene["data"].append(p)

    scene["data"].append(
        tlm.render_rays(P + ray_render_start*V, P + ray_render_end*V, layer=0)
    )
    
    # Display
    scene["controls"] = {"show_bounding_cylinders": True, "show_optical_axis": True, "show_other_axes": True}
    tlm.display_scene(scene)


demo()